In [26]:
import pandas as pd

In [27]:
df=pd.read_csv("dataset.csv")

In [28]:
df.dtypes

id                     int64
title                 object
genre                 object
original_language     object
overview              object
popularity           float64
release_date          object
vote_average         float64
vote_count             int64
dtype: object

In [29]:
df.head()

,id,title,genre,original_language,overview,popularity,release_date,vote_average,vote_count
0,278,The Shawshank Redemption,"Drama,Crime",en,Framed in the 1940s for the double murder of h...,94.075,1994-09-23,8.7,21862
1,19404,Dilwale Dulhania Le Jayenge,"Comedy,Drama,Romance",hi,"Raj is a rich, carefree, happy-go-lucky second...",25.408,1995-10-19,8.7,3731
2,238,The Godfather,"Drama,Crime",en,"Spanning the years 1945 to 1955, a chronicle o...",90.585,1972-03-14,8.7,16280
3,424,Schindler's List,"Drama,History,War",en,The true story of how businessman Oskar Schind...,44.761,1993-12-15,8.6,12959
4,240,The Godfather: Part II,"Drama,Crime",en,In the continuing saga of the Corleone crime f...,57.749,1974-12-20,8.6,9811


In [30]:
df.describe()

,id,popularity,vote_average,vote_count
count,10000.000000,10000.000000,10000.000000,10000.000000
mean,161243.505000,34.697267,6.621150,1547.309400
std,211422.046043,211.684175,0.766231,2648.295789
min,5.000000,0.600000,4.600000,200.000000
25%,10127.750000,9.154750,6.100000,315.000000
50%,30002.500000,13.637500,6.600000,583.500000
75%,310133.500000,25.651250,7.200000,1460.000000
max,934761.000000,10436.917000,8.700000,31917.000000


In [31]:
print("-"*120)

------------------------------------------------------------------------------------------------------------------------


In [32]:
import os, sys
from pyspark.sql import SparkSession
from pyspark.sql.functions import *

In [33]:
spark = SparkSession.builder.master("local[2]").appName("Netflix Movie Recommender").getOrCreate()

In [34]:
spark

In [35]:
movies=spark.read.csv("dataset.csv",header=True)


In [36]:
movies.count()

10005

In [37]:
movies.show()

+------+--------------------+--------------------+-----------------+--------------------+----------+------------+------------+----------+
|    id|               title|               genre|original_language|            overview|popularity|release_date|vote_average|vote_count|
+------+--------------------+--------------------+-----------------+--------------------+----------+------------+------------+----------+
|   278|The Shawshank Red...|         Drama,Crime|               en|Framed in the 194...|    94.075|  1994-09-23|         8.7|     21862|
| 19404|Dilwale Dulhania ...|Comedy,Drama,Romance|               hi|Raj is a rich, ca...|    25.408|  1995-10-19|         8.7|      3731|
|   238|       The Godfather|         Drama,Crime|               en|Spanning the year...|    90.585|  1972-03-14|         8.7|     16280|
|   424|    Schindler's List|   Drama,History,War|               en|The true story of...|    44.761|  1993-12-15|         8.6|     12959|
|   240|The Godfather: Pa...|     

In [38]:
movies.describe().show()

+-------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+------------------+
|summary|                  id|               title|               genre|   original_language|            overview|          popularity|        release_date|        vote_average|        vote_count|
+-------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+------------------+
|  count|               10005|               10005|               10002|               10005|                9992|                9999|                9999|                9997|              9997|
|   mean|          161243.505|  1172.9382272727273|                null|               6.994|              1383.0|   34.95272369243594|  38.651246376811606|   6.759508516568501|1514.8023986727917|
| stddev|   211

In [39]:
movies.printSchema()

root
 |-- id: string (nullable = true)
 |-- title: string (nullable = true)
 |-- genre: string (nullable = true)
 |-- original_language: string (nullable = true)
 |-- overview: string (nullable = true)
 |-- popularity: string (nullable = true)
 |-- release_date: string (nullable = true)
 |-- vote_average: string (nullable = true)
 |-- vote_count: string (nullable = true)



In [40]:
movies.count(),len(movies.columns)

(10005, 9)

In [41]:
movies.columns


['id',
 'title',
 'genre',
 'original_language',
 'overview',
 'popularity',
 'release_date',
 'vote_average',
 'vote_count']

In [42]:
movies=movies.select("id","title","genre","overview")

In [43]:
movies.show()

+------+--------------------+--------------------+--------------------+
|    id|               title|               genre|            overview|
+------+--------------------+--------------------+--------------------+
|   278|The Shawshank Red...|         Drama,Crime|Framed in the 194...|
| 19404|Dilwale Dulhania ...|Comedy,Drama,Romance|Raj is a rich, ca...|
|   238|       The Godfather|         Drama,Crime|Spanning the year...|
|   424|    Schindler's List|   Drama,History,War|The true story of...|
|   240|The Godfather: Pa...|         Drama,Crime|In the continuing...|
|667257|   Impossible Things|        Family,Drama|Matilde is a woma...|
|   129|       Spirited Away|Animation,Family,...|A young girl, Chi...|
|730154|      Your Eyes Tell|       Romance,Drama|A tragic accident...|
|372754|Dou kyu sei – Cla...|   Romance,Animation|Rihito Sajo, an h...|
|372058|          Your Name.|Romance,Animation...|High schoolers Mi...|
|   389|        12 Angry Men|               Drama|The defense an

In [44]:
movies.select([count(when(col(columns).isNull(), columns)).alias(columns) for columns in movies.columns]).show()

+---+-----+-----+--------+
| id|title|genre|overview|
+---+-----+-----+--------+
|  0|    0|    3|      13|
+---+-----+-----+--------+



In [45]:
df=movies.select("id","title",concat(movies.overview, movies.genre).alias("tags"))


In [46]:
df.show()


+------+--------------------+--------------------+
|    id|               title|                tags|
+------+--------------------+--------------------+
|   278|The Shawshank Red...|Framed in the 194...|
| 19404|Dilwale Dulhania ...|Raj is a rich, ca...|
|   238|       The Godfather|Spanning the year...|
|   424|    Schindler's List|The true story of...|
|   240|The Godfather: Pa...|In the continuing...|
|667257|   Impossible Things|Matilde is a woma...|
|   129|       Spirited Away|A young girl, Chi...|
|730154|      Your Eyes Tell|A tragic accident...|
|372754|Dou kyu sei – Cla...|Rihito Sajo, an h...|
|372058|          Your Name.|High schoolers Mi...|
|   389|        12 Angry Men|The defense and t...|
|696374|   Gabriel's Inferno|An intriguing and...|
|496243|            Parasite|All unemployed, K...|
|   497|      The Green Mile|A supernatural ta...|
|724089|Gabriel's Inferno...|Professor Gabriel...|
|   155|     The Dark Knight|Batman raises the...|
|   429|The Good, the Bad...|Wh

In [47]:
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer


In [48]:
from sklearn.feature_extraction.text import CountVectorizer

In [49]:

df_=df.toPandas()

In [50]:
type(df_["tags"][0])

str

In [51]:
tokens=[]
for i in range(len(df_)):
    tokens.append(word_tokenize(str(df_["tags"][i])))
    

In [52]:
tokens[3]

['The',
 'true',
 'story',
 'of',
 'how',
 'businessman',
 'Oskar',
 'Schindler',
 'saved',
 'over',
 'a',
 'thousand',
 'Jewish',
 'lives',
 'from',
 'the',
 'Nazis',
 'while',
 'they',
 'worked',
 'as',
 'slaves',
 'in',
 'his',
 'factory',
 'during',
 'World',
 'War',
 'II.Drama',
 ',',
 'History',
 ',',
 'War']

In [53]:
stopwordsList=stopwords.words("english")
stopwordsList.extend([',','.','-','--','!'])

wordList=[]
for tokenList in tokens:
    words=[]
    for word in tokenList:
        if word.lower() not in stopwordsList:
            words.append(word.lower())
            
    wordList.append(words)

wordList[0]

['framed',
 '1940s',
 'double',
 'murder',
 'wife',
 'lover',
 'upstanding',
 'banker',
 'andy',
 'dufresne',
 'begins',
 'new',
 'life',
 'shawshank',
 'prison',
 'puts',
 'accounting',
 'skills',
 'work',
 'amoral',
 'warden',
 'long',
 'stretch',
 'prison',
 'dufresne',
 'comes',
 'admired',
 'inmates',
 'including',
 'older',
 'prisoner',
 'named',
 'red',
 'integrity',
 'unquenchable',
 'sense',
 'hope.drama',
 'crime']

In [54]:
wnet=WordNetLemmatizer()

for i in range(len(wordList)):
    for j in range(len(wordList[i])):
        wordList[i][j]=wnet.lemmatize(wordList[i][j],pos='v')
        
wordList[0]

['frame',
 '1940s',
 'double',
 'murder',
 'wife',
 'lover',
 'upstanding',
 'banker',
 'andy',
 'dufresne',
 'begin',
 'new',
 'life',
 'shawshank',
 'prison',
 'put',
 'account',
 'skills',
 'work',
 'amoral',
 'warden',
 'long',
 'stretch',
 'prison',
 'dufresne',
 'come',
 'admire',
 'inmates',
 'include',
 'older',
 'prisoner',
 'name',
 'red',
 'integrity',
 'unquenchable',
 'sense',
 'hope.drama',
 'crime']

In [55]:
for i in range(len(tokens)):
    tokens[i]=' '.join(wordList[i])

In [56]:
tokens[0]

'frame 1940s double murder wife lover upstanding banker andy dufresne begin new life shawshank prison put account skills work amoral warden long stretch prison dufresne come admire inmates include older prisoner name red integrity unquenchable sense hope.drama crime'

In [57]:
cv = CountVectorizer(max_features=10005)

In [58]:
cv

CountVectorizer(max_features=10005)

In [59]:
vector=cv.fit_transform(tokens).toarray()

In [60]:
vector

array([[0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       ...,
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0]], dtype=int64)

In [61]:
vector.shape

(10005, 10005)

In [62]:
from sklearn.metrics.pairwise import cosine_similarity

In [63]:
similarity=cosine_similarity(vector)

In [64]:
similarity

array([[1.        , 0.05143445, 0.1228683 , ..., 0.06666667, 0.10660036,
        0.06299408],
       [0.05143445, 1.        , 0.06825236, ..., 0.        , 0.03289758,
        0.        ],
       [0.1228683 , 0.06825236, 1.        , ..., 0.02948839, 0.06286946,
        0.08359173],
       ...,
       [0.06666667, 0.        , 0.02948839, ..., 1.        , 0.08528029,
        0.03779645],
       [0.10660036, 0.03289758, 0.06286946, ..., 0.08528029, 1.        ,
        0.04029115],
       [0.06299408, 0.        , 0.08359173, ..., 0.03779645, 0.04029115,
        1.        ]])

In [65]:
df_[df_['title']=="The Godfather"].index[0]

2

In [66]:
def recommand(movies):
    index=df_[df_['title']==movies].index[0]
    distance = sorted(list(enumerate(similarity[index])), reverse=True, key=lambda vector:vector[1])
    for i in distance[0:11]:
        print(df_.iloc[i[0]].title)

In [67]:
import pickle


In [71]:
pickle.dump(df_.to_dict(), open('movies_dict.pkl', 'wb'))


In [69]:
pickle.dump(similarity, open('similarity.pkl', 'wb'))


In [70]:
pickle.load(open('movies_list.pkl', 'rb'))

,id,title,tags
0,278,The Shawshank Redemption,Framed in the 1940s for the double murder of h...
1,19404,Dilwale Dulhania Le Jayenge,"Raj is a rich, carefree, happy-go-lucky second..."
2,238,The Godfather,"Spanning the years 1945 to 1955, a chronicle o..."
3,424,Schindler's List,The true story of how businessman Oskar Schind...
4,240,The Godfather: Part II,In the continuing saga of the Corleone crime f...
...,...,...,...
10000,10196,The Last Airbender,"The story follows the adventures of Aang, a yo..."
10001,331446,Sharknado 3: Oh Hell No!,The sharks take bite out of the East Coast whe...
10002,13995,Captain America,"""During World War II, a brave, patriotic Ameri..."
10003,2312,In the Name of the King: A Dungeon Siege Tale,A man named Farmer sets out to rescue his kidn...
